In [1]:
from dash import Dash, dcc, html, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

In [2]:
external_CSS = [dbc.themes.MINTY, 'https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=[external_CSS])


# COMPONENTS
mytext = dcc.Markdown(children="# This is a Header1")
myinput = dbc.Input(value = "## This is input")


# GRAPH COMPONENT
sampledata = px.data.medals_long()
mytitle = dcc.Markdown(children="# Graph title goes here")
mygraph = dcc.Graph(figure={})
dropdown = dcc.Dropdown(options=['Bar Plot', 'Scatter Plot'],
                         value = 'Bar Plot',
                         clearable=False)

# CHLORPETH COMPONENT
sampleCSV = pd.read_csv("https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Good_to_Know/Dash2.0/social_capital.csv")
chlr_title = dcc.Markdown(children='# US Map Data')
chlr_graph = dcc.Graph(figure={})
chlr_dropdown = dcc.Dropdown(
    options=sampleCSV.columns.values[2:],
    value='Cesarean Delivery Rate',
    clearable=False
)

# HOVER COMPONENT


# LAYOUT
page_items = [mytext, myinput]
page_items = [mytitle, dropdown, mygraph]
page_items = [
    dbc.Row(dbc.Col([chlr_title], width=6), justify='center'), 
    dbc.Row(dbc.Col([chlr_graph], width=12)),
    dbc.Row(dbc.Col([chlr_dropdown], width=6), justify='center')
]
app.layout = dbc.Container(page_items)
sampleCSV.head()


# HTML/CSS STYLED GRAPHS
sample_2 = px.data.gapminder()
app.layout = html.Div(
    [dcc.Dropdown(id='dpdn2', value=['Germany', 'Brazil'], multi=True, 
                  options=[{'label': x, 'value': x} for x in
                           sample_2.country.unique()]),
     html.Div([
         dcc.Graph(id='pie-graph', figure={}, className='six colummns'),
         dcc.Graph(id='my-graph', figure={}, clickData=None, hoverData=None,
                    className='six colummns',
                    config={
                       'staticPlot': False,
                       'scrollZoom': True,
                       'doubleClick': 'reset',
                       'showTips': False,
                       'displayModeBar': True,
                       'watermark': True
                   })
     ])
    ]
 )


In [3]:
# CALLBACK AND ITS FUNCTION
# @app.callback(
#     Output(mytext, component_property='children'),
#     Input(myinput, component_property='value')
# )
# def update_title(user_input):
#     return user_input

# GRAPH CALLBACK
# @app.callback(
#     Output(mygraph, component_property='figure'),
#     Input(dropdown, component_property='value')
# )
# def update_graph(user_input):
#     if user_input == 'Bar Plot':
#         fig = px.bar(data_frame = sampledata, x='nation', y='count', color='medal')
#     elif user_input == 'Scatter Plot':
#         fig = px.scatter(data_frame = sampledata, x='nation', y='count', color='medal', symbol='medal')
#     return fig


# MAP GRAPH CALLBACK
# @app.callback(
#     Output(chlr_title, component_property='children'),
#     Output(chlr_graph, component_property='figure'),
#     Input(chlr_dropdown, component_property='value')
# )
# def update_map(column_name):
#     fig = px.choropleth(
#         data_frame = sampleCSV,
#         locations = 'STATE',
#         locationmode = 'USA-states',
#         scope = 'usa',
#         height = 600,
#         color = column_name,
#         animation_frame = 'YEAR'
#     )
#     return '# ' + column_name, fig

# HTML GRAPHS CALLBACK
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='dpdn2', component_property='value')
)
def update_line_graph(country_chosen):
    fig = px.line(data_frame=sample_2, x='year', y='gdpPercap', color='country',
                  custom_data=['country', 'continent', 'lifeExp', 'pop'])
    fig.update_traces(mode='lines+markers')
    return fig
@app.callback(
    Output(component_id='pie-graph', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, country_chosen):
        if clk_data is None:
            dff2 = sample_2[sample_2.country.isin(country_chosen)]
            dff2 = dff2[dff2.year == 1952]
            print(dff2)
            fig2 = px.pie(data_frame=dff2, values='pop', names='country', title='Pop for 1952')
            return fig2
        else:
            print(f'clk_data: {clk_data}')
            dff2 = sample_2[sample_2.country.isin(country_chosen)]
            clk_year = clk_data['points'][0]['x'] 
            dff2 = dff2[dff2.year == clk_year]
            fig2 = px.pie(data_frame=dff2, values='pop', names='country', title='Pop for 1952')
            return fig2

In [4]:
if __name__ == '__main__':
    app.run_server(port=8053)

OSError: Address 'http://127.0.0.1:8053' already in use.
    Try passing a different port to run_server.